# Popcorn Hacks

When you run the test case with find_mode(pd.Series([1, 2, 2, 3, 4, 2, 5])), it should return 2, which appears three times in the Series and is the most common value.

In [ ]:
# Fire Prediction Data Analysis Using Satellite Data
# Student: [Your Name]

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from scipy.stats import pearsonr

# 1. Load the satellite data
# For this example, I'm assuming we have a CSV file with satellite fire data
# In a real scenario, you would replace 'fire_satellite_data.csv' with your actual file
print("Task 1: Loading the data")

# Create sample data since we don't have the actual CSV
# This simulates what the data might look like for fire incidents detected via satellite
np.random.seed(42)  # For reproducibility

# Create sample data
n_samples = 200
vegetation_types = ['Forest', 'Grassland', 'Shrubland', 'Wetland', 'Urban']
weather_conditions = ['Clear', 'Cloudy', 'Windy', 'Dry', 'Humid']

data = {
    'incident_id': range(1, n_samples + 1),
    'latitude': np.random.uniform(32.0, 42.0, n_samples),
    'longitude': np.random.uniform(-124.0, -114.0, n_samples),
    'temperature_f': np.random.normal(95, 15, n_samples),  # Normal distribution around 95°F
    'wind_speed_mph': np.random.gamma(2, 5, n_samples),  # Gamma distribution for wind speed
    'humidity_pct': np.random.uniform(10, 70, n_samples),
    'vegetation_type': np.random.choice(vegetation_types, n_samples),
    'weather_condition': np.random.choice(weather_conditions, n_samples),
    'fire_intensity': np.random.gamma(3, 2, n_samples),  # Fire intensity (MW/m²)
    'date_detected': pd.date_range(start='2023-05-01', periods=n_samples).strftime('%Y-%m-%d').tolist()
}

# Create DataFrame
df = pd.DataFrame(data)

# Modify data to create realistic relationships
# Higher temperatures in grasslands
df.loc[df['vegetation_type'] == 'Grassland', 'temperature_f'] += 8
# Higher fire intensity in dry conditions
df.loc[df['weather_condition'] == 'Dry', 'fire_intensity'] *= 1.5
# Higher wind speeds increase fire intensity
df['fire_intensity'] += df['wind_speed_mph'] * 0.1
# Lower humidity increases fire intensity
df['fire_intensity'] += (70 - df['humidity_pct']) * 0.05

# View the first few rows of the dataset
print("Sample of the dataset:")
print(df.head())

# Get basic information about the dataset
print("\nDataset Information:")
print(df.info())

# Generate basic statistics
print("\nBasic Statistics:")
print(df.describe())

# 2. Implementing the required algorithms

print("\nTask 2: Implementing Algorithms")

# Find fire incidents with highest and lowest overall average temperature
print("\nHighest and Lowest Temperature Incidents:")
highest_temp_incident = df.loc[df['temperature_f'].idxmax()]
lowest_temp_incident = df.loc[df['temperature_f'].idxmin()]

print(f"Highest temperature incident (ID: {highest_temp_incident['incident_id']}):")
print(f"  Temperature: {highest_temp_incident['temperature_f']:.2f}°F")
print(f"  Location: ({highest_temp_incident['latitude']:.4f}, {highest_temp_incident['longitude']:.4f})")
print(f"  Vegetation: {highest_temp_incident['vegetation_type']}")

print(f"\nLowest temperature incident (ID: {lowest_temp_incident['incident_id']}):")
print(f"  Temperature: {lowest_temp_incident['temperature_f']:.2f}°F")
print(f"  Location: ({lowest_temp_incident['latitude']:.4f}, {lowest_temp_incident['longitude']:.4f})")
print(f"  Vegetation: {lowest_temp_incident['vegetation_type']}")

# Calculate temperature range (difference between max and min temp)
# For this task, we'll consider daily temperature fluctuations by creating a min and max temperature
# Let's create these columns first
df['min_temperature_f'] = df['temperature_f'] - np.random.uniform(5, 15, n_samples)
df['max_temperature_f'] = df['temperature_f'] + np.random.uniform(5, 15, n_samples)

# Now calculate the temperature range for each incident
df['temperature_range'] = df['max_temperature_f'] - df['min_temperature_f']

print("\nTemperature Range Statistics:")
print(f"Average temperature range: {df['temperature_range'].mean():.2f}°F")
print(f"Maximum temperature range: {df['temperature_range'].max():.2f}°F")
print(f"Minimum temperature range: {df['temperature_range'].min():.2f}°F")

# Identify fire incidents where temperature exceeded the average
avg_temp = df['temperature_f'].mean()
high_temp_incidents = df[df['temperature_f'] > avg_temp]

print(f"\nIncidents with Above-Average Temperatures (>{avg_temp:.2f}°F):")
print(f"Count: {high_temp_incidents.shape[0]} incidents ({high_temp_incidents.shape[0]/n_samples*100:.1f}% of total)")
print(f"Average fire intensity in high-temperature incidents: {high_temp_incidents['fire_intensity'].mean():.2f}")
print(f"Average fire intensity in other incidents: {df[df['temperature_f'] <= avg_temp]['fire_intensity'].mean():.2f}")

# Group fire incidents by vegetation type and weather conditions
grouped_data = df.groupby(['vegetation_type', 'weather_condition']).agg({
    'temperature_f': 'mean',
    'wind_speed_mph': 'mean',
    'fire_intensity': 'mean',
    'incident_id': 'count'
}).rename(columns={'incident_id': 'count'}).reset_index()

print("\nFire Incidents Grouped by Vegetation Type and Weather Condition:")
print(grouped_data)

# 3. Answer analytical questions

print("\nTask 3: Analytical Questions")

# Question 1: Correlation between vegetation type and fire intensity
print("\nQuestion 1: Correlation between vegetation type and fire intensity")

# One-hot encode vegetation types for correlation analysis
veg_dummies = pd.get_dummies(df['vegetation_type'], prefix='veg')
df_with_dummies = pd.concat([df, veg_dummies], axis=1)

# Calculate correlation between each vegetation type and fire intensity
veg_correlations = {}
for veg_type in vegetation_types:
    column = f'veg_{veg_type}'
    correlation, p_value = pearsonr(df_with_dummies[column], df_with_dummies['fire_intensity'])
    veg_correlations[veg_type] = correlation

# Create a DataFrame for visualization
veg_corr_df = pd.DataFrame(list(veg_correlations.items()), columns=['Vegetation Type', 'Correlation with Fire Intensity'])
print(veg_corr_df)

# Visualize correlation
plt.figure(figsize=(10, 6))
sns.barplot(x='Vegetation Type', y='Correlation with Fire Intensity', data=veg_corr_df)
plt.title('Correlation between Vegetation Type and Fire Intensity')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('vegetation_fire_correlation.png')
plt.close()

# Question 2: Weather condition with highest average fire intensity
print("\nQuestion 2: Weather condition with highest average fire intensity")

weather_intensity = df.groupby('weather_condition')['fire_intensity'].mean().reset_index()
weather_intensity = weather_intensity.sort_values('fire_intensity', ascending=False)
print(weather_intensity)

highest_intensity_weather = weather_intensity.iloc[0]['weather_condition']
highest_intensity_value = weather_intensity.iloc[0]['fire_intensity']

print(f"\nThe weather condition with the highest average fire intensity is '{highest_intensity_weather}'")
print(f"with an average intensity of {highest_intensity_value:.2f} MW/m²")

# Visualize average fire intensity by weather condition
plt.figure(figsize=(10, 6))
sns.barplot(x='weather_condition', y='fire_intensity', data=df, estimator=np.mean, ci=None)
plt.title('Average Fire Intensity by Weather Condition')
plt.xlabel('Weather Condition')
plt.ylabel('Fire Intensity (MW/m²)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('weather_fire_intensity.png')
plt.close()

# Question 3: Percentage of fire incidents with temperatures above 100°F
print("\nQuestion 3: Percentage of fire incidents with temperatures above 100°F")

high_temp_count = df[df['temperature_f'] > 100].shape[0]
total_count = df.shape[0]
high_temp_percentage = (high_temp_count / total_count) * 100

print(f"Number of incidents with temperature above 100°F: {high_temp_count}")
print(f"Total number of incidents: {total_count}")
print(f"Percentage: {high_temp_percentage:.2f}%")

# Visualize temperature distribution with 100°F threshold marked
plt.figure(figsize=(10, 6))
sns.histplot(df['temperature_f'], bins=25, kde=True)
plt.axvline(x=100, color='red', linestyle='--', label='100°F Threshold')
plt.title('Distribution of Fire Incident Temperatures')
plt.xlabel('Temperature (°F)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('temperature_distribution.png')
plt.close()

# 4. Create a SQLite database

print("\nTask 4: SQLite Database Analysis")

# Connect to SQLite database (will be created if it doesn't exist)
conn = sqlite3.connect('fire_incidents.db')
cursor = conn.cursor()

# Create fire_incidents table
cursor.execute('''
CREATE TABLE IF NOT EXISTS fire_incidents (
    incident_id INTEGER PRIMARY KEY,
    latitude REAL,
    longitude REAL,
    temperature_f REAL,
    wind_speed_mph REAL,
    humidity_pct REAL,
    vegetation_type TEXT,
    weather_condition TEXT,
    fire_intensity REAL,
    date_detected TEXT,
    min_temperature_f REAL,
    max_temperature_f REAL,
    temperature_range REAL
)
''')

# Insert the data into the table
df.to_sql('fire_incidents', conn, if_exists='replace', index=False)

print("Fire incidents data stored in SQLite database")

# Execute SQL queries

# SQL Query 1: Average temperature and wind speed for each vegetation type
print("\nSQL Query 1: Average temperature and wind speed by vegetation type")
query1 = '''
SELECT vegetation_type, 
       ROUND(AVG(temperature_f), 2) AS avg_temperature,
       ROUND(AVG(wind_speed_mph), 2) AS avg_wind_speed,
       ROUND(AVG(fire_intensity), 2) AS avg_fire_intensity,
       COUNT(*) AS incident_count
FROM fire_incidents
GROUP BY vegetation_type
ORDER BY avg_fire_intensity DESC
'''

sql_result1 = pd.read_sql_query(query1, conn)
print(sql_result1)

# SQL Query 2: Fire incidents with temp > 120°F and wind speed > 15 mph
print("\nSQL Query 2: High-risk incidents (temp > 120°F and wind > 15 mph)")
query2 = '''
SELECT incident_id, temperature_f, wind_speed_mph, vegetation_type, weather_condition, fire_intensity
FROM fire_incidents
WHERE temperature_f > 120 AND wind_speed_mph > 15
ORDER BY fire_intensity DESC
'''

sql_result2 = pd.read_sql_query(query2, conn)
print(sql_result2)

# SQL Query 3: Average fire intensity by weather condition
print("\nSQL Query 3: Average fire intensity by weather condition")
query3 = '''
SELECT weather_condition,
       ROUND(AVG(temperature_f), 2) AS avg_temperature,
       ROUND(AVG(fire_intensity), 2) AS avg_fire_intensity,
       COUNT(*) AS incident_count
FROM fire_incidents
GROUP BY weather_condition
ORDER BY avg_fire_intensity DESC
'''

sql_result3 = pd.read_sql_query(query3, conn)
print(sql_result3)

# Compare SQL vs Pandas approaches
print("\nComparison: SQL vs. Pandas Approaches")

# Example comparison: Average fire intensity by weather condition
# Pandas approach (already done above)
pandas_start_time = pd.Timestamp.now()
pandas_result = df.groupby('weather_condition')['fire_intensity'].mean().reset_index()
pandas_end_time = pd.Timestamp.now()
pandas_duration = (pandas_end_time - pandas_start_time).total_seconds()

# SQL approach (already done above)
sql_start_time = pd.Timestamp.now()
sql_result = pd.read_sql_query('''
    SELECT weather_condition, AVG(fire_intensity) as avg_fire_intensity
    FROM fire_incidents
    GROUP BY weather_condition
''', conn)
sql_end_time = pd.Timestamp.now()
sql_duration = (sql_end_time - sql_start_time).total_seconds()

print(f"Pandas execution time: {pandas_duration:.6f} seconds")
print(f"SQL execution time: {sql_duration:.6f} seconds")

print('''
Comparison of SQL vs. Pandas Approaches:

1. Performance:
   - For this small dataset, performance differences are minimal
   - Pandas is typically faster for in-memory operations
   - SQL can be more efficient for very large datasets that don't fit in memory
   - SQL databases offer indexing which can significantly speed up queries on large datasets

2. Readability:
   - SQL queries can be more readable for people familiar with SQL
   - Pandas code is more intuitive for Python programmers
   - SQL is declarative (what you want), while Pandas is more procedural (how to get it)

3. Ease of Use:
   - Pandas offers more flexibility for complex data manipulations
   - Pandas integrates better with visualization libraries and machine learning frameworks
   - SQL is better for data integrity constraints and multi-user access
   - SQL is universal across many programming languages

4. Other Considerations:
   - SQL is better for data persistence and sharing across applications
   - Pandas is better for exploratory data analysis and prototyping
   - SQL requires database setup and management
   - Combining both approaches often yields the best results
''')

# Close the database connection
conn.close()

# Summary visualizations

# Create a correlation matrix for numerical variables
plt.figure(figsize=(10, 8))
correlation_matrix = df[['temperature_f', 'wind_speed_mph', 'humidity_pct', 'fire_intensity']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Fire Incident Variables')
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.close()

# Create a scatter plot of temperature vs fire intensity colored by vegetation type
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df, x='temperature_f', y='fire_intensity', hue='vegetation_type', alpha=0.7)
plt.title('Relationship Between Temperature and Fire Intensity by Vegetation Type')
plt.xlabel('Temperature (°F)')
plt.ylabel('Fire Intensity (MW/m²)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(title='Vegetation Type')
plt.tight_layout()
plt.savefig('temp_intensity_scatter.png')
plt.close()

# Create boxplots for fire intensity by vegetation type
plt.figure(figsize=(12, 8))
sns.boxplot(x='vegetation_type', y='fire_intensity', data=df)
plt.title('Fire Intensity Distribution by Vegetation Type')
plt.xlabel('Vegetation Type')
plt.ylabel('Fire Intensity (MW/m²)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('intensity_boxplot.png')
plt.close()

print("\nAnalysis complete! All visualizations have been saved.")